# Calculate normalised enrichment scores of hallmarks

Script related to figure 1d, e

### Aim:
- Calculate NES of Hallmark Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Install packages

In [ ]:
#source("https://bioconductor.org/biocLite.R")

In [ ]:
#biocLite("GSVA")

In [ ]:
#biocLite("BBmisc")

In [ ]:
#biocLite("nortest")

## Load packages

In [3]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/1/")

In [2]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



## Load expression data

In [4]:
data <- read.csv("~/git_repos/HGSOC_TME_Heterogeneity/Data/1/TreatmentNaive_log2exp_loess_norm.txt",
                 sep='\t',
                 header=T,
                 row.names='Hugo_Symbol')

In [5]:
head(data)

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
A1BG,4.124711,3.880633,4.864973,4.167882,4.317845,3.679057,4.092837,4.370571,4.252637,4.174694,⋯,3.804537,4.416981,4.169014,4.503313,4.146333,4.078398,4.011369,3.845128,3.600927,4.521857
A2BP1,3.960924,4.033715,3.721251,3.916073,4.338194,3.357396,3.872447,3.803388,3.484735,4.427405,⋯,4.991068,3.799913,4.117883,4.122134,4.221760,3.648326,3.675317,3.546925,3.788279,4.920841
A2M,12.603155,10.859838,12.612923,9.176999,10.548790,11.840776,7.200547,11.835658,11.535547,7.080118,⋯,9.061598,9.189459,8.844728,7.464440,6.726233,9.265911,10.740486,9.735582,10.781599,8.734304
A2ML1,3.945877,4.000621,3.767932,3.789729,3.543204,3.805520,4.336280,4.031757,4.166018,3.673146,⋯,4.231174,3.871651,3.287232,3.721093,3.723801,3.884499,3.694111,3.937682,3.600471,3.402861
A4GALT,3.597277,3.863757,4.051844,4.113169,3.975144,3.991293,5.439960,4.696177,4.171788,4.041633,⋯,4.183089,4.699729,4.019537,3.739865,4.591953,4.997860,4.099630,5.001239,4.714710,4.145119
A4GNT,3.714481,3.593169,3.241352,3.515850,3.364447,3.767865,3.768695,3.163977,3.423377,3.205154,⋯,3.896812,3.915653,3.351422,3.719759,3.641028,3.114521,3.560199,3.539738,3.760903,3.479266


In [6]:
data_mtx <- as.matrix(data)

## Load hallmark gene sets

In [11]:
Hallmark_geneSets <- read.csv("~/git_repos/HGSOC_TME_Heterogeneity/Data/1/HallmarksStromaImmune_GeneSets.txt",
                              sep='\t',
                              header=F,
                              row.names=1)

In [12]:
head(Hallmark_geneSets)

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,⋯,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202
HALLMARK_TNFA_SIGNALING_VIA_NFKB,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_TNFA_SIGNALING_VIA_NFKB,JUNB,CXCL2,ATF3,NFKBIA,TNFAIP3,PTGS2,CXCL1,IER3,CD83,⋯,EIF1,BMP2,DUSP4,PDLIM5,ICOSLG,GFPT2,KLF2,TNC,SERPINB8,MXD1
HALLMARK_HYPOXIA,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_HYPOXIA,PGK1,PDK1,GBE1,PFKL,ALDOA,ENO2,PGM1,NDRG1,HK2,⋯,HDLBP,ILVBL,NCAN,TGM2,ETS1,HOXB9,SELENBP1,FOSL2,SULT2B1,TGFB3
HALLMARK_CHOLESTEROL_HOMEOSTASIS,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_CHOLESTEROL_HOMEOSTASIS,FDPS,CYP51A1,IDI1,FDFT1,DHCR7,SQLE,HMGCS1,NSDHL,LSS,⋯,,,,,,,,,,
HALLMARK_MITOTIC_SPINDLE,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_MITOTIC_SPINDLE,ARHGEF2,CLASP1,KIF11,KIF23,ALS2,ARF6,MYO9B,MYH9,TUBGCP3,⋯,RHOT2,SORBS2,CDC42EP1,VCL,CLIP1,STK38L,YWHAE,RAPGEF5,CEP72,CSNK1D
HALLMARK_WNT_BETA_CATENIN_SIGNALING,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_WNT_BETA_CATENIN_SIGNALING,MYC,CTNNB1,JAG2,NOTCH1,DLL1,AXIN2,PSEN2,FZD1,NOTCH4,⋯,,,,,,,,,,
HALLMARK_TGF_BETA_SIGNALING,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_TGF_BETA_SIGNALING,TGFBR1,SMAD7,TGFB1,SMURF2,SMURF1,BMPR2,SKIL,SKI,ACVR1,⋯,,,,,,,,,,


### Remove website column

In [13]:
Hallmark_geneSets <- Hallmark_geneSets[-1]

In [14]:
head(Hallmark_geneSets)

,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202
HALLMARK_TNFA_SIGNALING_VIA_NFKB,JUNB,CXCL2,ATF3,NFKBIA,TNFAIP3,PTGS2,CXCL1,IER3,CD83,CCL20,⋯,EIF1,BMP2,DUSP4,PDLIM5,ICOSLG,GFPT2,KLF2,TNC,SERPINB8,MXD1
HALLMARK_HYPOXIA,PGK1,PDK1,GBE1,PFKL,ALDOA,ENO2,PGM1,NDRG1,HK2,ALDOC,⋯,HDLBP,ILVBL,NCAN,TGM2,ETS1,HOXB9,SELENBP1,FOSL2,SULT2B1,TGFB3
HALLMARK_CHOLESTEROL_HOMEOSTASIS,FDPS,CYP51A1,IDI1,FDFT1,DHCR7,SQLE,HMGCS1,NSDHL,LSS,MVD,⋯,,,,,,,,,,
HALLMARK_MITOTIC_SPINDLE,ARHGEF2,CLASP1,KIF11,KIF23,ALS2,ARF6,MYO9B,MYH9,TUBGCP3,CKAP5,⋯,RHOT2,SORBS2,CDC42EP1,VCL,CLIP1,STK38L,YWHAE,RAPGEF5,CEP72,CSNK1D
HALLMARK_WNT_BETA_CATENIN_SIGNALING,MYC,CTNNB1,JAG2,NOTCH1,DLL1,AXIN2,PSEN2,FZD1,NOTCH4,LEF1,⋯,,,,,,,,,,
HALLMARK_TGF_BETA_SIGNALING,TGFBR1,SMAD7,TGFB1,SMURF2,SMURF1,BMPR2,SKIL,SKI,ACVR1,PMEPA1,⋯,,,,,,,,,,


In [15]:
Hallmark_geneSets <- convertRowsToList(Hallmark_geneSets)

## Run ssGSEA

In [16]:
hsi_NES <- gsva(data_mtx,
                Hallmark_geneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Estimating ssGSEA scores for 52 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [17]:
hsi_NES <- data.frame("Term"=rownames(hsi_NES),
                      hsi_NES)

rownames(hsi_NES) <- NULL

In [18]:
head(hsi_NES)

Term,A01,A02,A03,A04,A05,A06,A07,A08,A09,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.1365760,0.1622527,0.1679446,0.1658720,0.1082707,0.1033835,0.0683624,0.07016183,0.1115274,⋯,0.1281450,0.2591543,0.2156885,0.1126508,0.2002938,0.2419946,0.1914320,0.1947142,0.1893358,0.06965322
HALLMARK_HYPOXIA,0.1777680,0.2182873,0.2066369,0.2280754,0.2133397,0.1951075,0.1847733,0.17668794,0.2085090,⋯,0.2329528,0.2851864,0.2588453,0.2423339,0.2801138,0.2631574,0.2612868,0.2541697,0.2550989,0.17013909
HALLMARK_CHOLESTEROL_HOMEOSTASIS,0.2910560,0.3071711,0.3040415,0.3029025,0.2912912,0.3026668,0.2503074,0.30745975,0.3366550,⋯,0.2978222,0.3115737,0.3149536,0.3273641,0.3572007,0.3259694,0.3283077,0.3103078,0.3337639,0.30601647
HALLMARK_MITOTIC_SPINDLE,0.2913235,0.3417554,0.3326834,0.3580851,0.3194368,0.3505266,0.4386014,0.35965903,0.3657099,⋯,0.3476544,0.3396198,0.3564771,0.3548904,0.3991286,0.3552740,0.3375994,0.3293698,0.3324187,0.31162160
HALLMARK_WNT_BETA_CATENIN_SIGNALING,0.1512311,0.1841402,0.1867324,0.2632747,0.2154267,0.2244500,0.2460888,0.24303028,0.2285515,⋯,0.1262571,0.1893952,0.1804980,0.1666257,0.2367696,0.1856328,0.1475494,0.1423987,0.1268914,0.11033910
HALLMARK_TGF_BETA_SIGNALING,0.3601273,0.4052729,0.3830989,0.4405695,0.3904583,0.3544433,0.3731478,0.35217498,0.3921855,⋯,0.4162216,0.4551743,0.4727299,0.3769614,0.4335947,0.4230834,0.4160767,0.4071078,0.4056907,0.32461787


In [19]:
write.table(hsi_NES,
            file="~/git_repos/HGSOC_TME_Heterogeneity/Data/1/HallmarksStromaImmune_NES.txt",
            sep='\t',
            row.names=F,
            col.names=T)

# End script